<a id='top'></a>
<img align="center" src="https://images.squarespace-cdn.com/content/v1/595b7bc8bebafb2819864641/1560334715445-7DNUAABRK8THY2UTJ5VW/ke17ZwdGBToddI8pDm48kFWxnDtCdRm2WA9rXcwtIYR7gQa3H78H3Y0txjaiv_0fDoOvxcdMmMKkDsyUqMSsMWxHk725yiiHCCLfrh8O1z5QPOohDIaIeljMHgDF5CVlOqpeNLcJ80NK65_fV7S1UcTSrQkGwCGRqSxozz07hWZrYGYYH8sg4qn8Lpf9k1pYMHPsat2_S1jaQY3SwdyaXg/Lrg_GDPR02.jpg" width="200" />
<p style="text-align: center;">
    <font size='5' type='bold'>
        <b>Защита данных клиентов</b>
    </font>
</p>

### Содержание

<a href='#section_0'>Описание проекта</a>

<a href='#section_1'>1. Загрузка данных</a>

<a href='#section_2'>2. Умножение матриц</a>

<a href='#section_3'>3. Алгоритм преобразования</a>

<a href='#section_4'>4. Проверка алгоритма</a>

<a href='#section_end'>Чек-лист готовности проекта</a>

<a id='section_0'></a>
# Описание проекта
Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Формулы, которые пригодятся нам в проекте:
* Предсказания:

$$
\begin{equation}\tag{1}a = X\cdot w\end{equation}
$$

* Задача обучения:

$$
\begin{equation}\tag{2}w = \arg\min_w MSE(X\cdot w, y)\end{equation}
$$

* Формула обучения:

$$
\begin{equation}\tag{3}w = (X^T \cdot X)^{-1} \cdot X^T\cdot  y\end{equation}
$$

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

<a href='#top'>Back to top</a>

<a id='section_1'></a>
## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
df

Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0       1     41.0   49600.0            1                  0
1       0     46.0   38000.0            1                  1
2       0     29.0   21000.0            0                  0
3       0     21.0   41700.0            2                  0
4       1     28.0   26100.0            0                  0
...   ...      ...       ...          ...                ...
4995    0     28.0   35700.0            2                  0
4996    0     34.0   52400.0            1                  0
4997    0     20.0   33900.0            2                  0
4998    1     22.0   32700.0            3                  0
4999    1     28.0   40600.0            1                  0

[5000 rows x 5 columns]

* Признаки: *Пол*, *Возраст* и *Зарплата* застрахованного, *Количество членов его семьи*.
* Целевой признак: *Количество страховых выплат* клиенту за последние 5 лет.

In [3]:
# Переименуем столбцы для удобства работы

df=df.rename(columns={'Пол': 'sex', 
                      'Возраст': 'age', 
                      'Зарплата': 'salary', 
                      'Члены семьи': 'family_members', 
                      'Страховые выплаты': 'insurance_compensations'}
            )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
sex                        5000 non-null int64
age                        5000 non-null float64
salary                     5000 non-null float64
family_members             5000 non-null int64
insurance_compensations    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков в данных нет. Посмотрим на дубликаты.

In [4]:
df.duplicated().sum()

153

In [5]:
# Избавимся от дубликатов

df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [6]:
# Посмотрим на общую информацию о данных

df.describe()

sex          age        salary  family_members  \
count  4847.000000  4847.000000   4847.000000     4847.000000   
mean      0.498453    31.023932  39895.811842        1.203425   
std       0.500049     8.487995   9972.953985        1.098664   
min       0.000000    18.000000   5300.000000        0.000000   
25%       0.000000    24.000000  33200.000000        0.000000   
50%       0.000000    30.000000  40200.000000        1.000000   
75%       1.000000    37.000000  46600.000000        2.000000   
max       1.000000    65.000000  79000.000000        6.000000   

       insurance_compensations  
count              4847.000000  
mean                  0.152259  
std                   0.468934  
min                   0.000000  
25%                   0.000000  
50%                   0.000000  
75%                   0.000000  
max                   5.000000

Данные подготовлены для дальнейшей работы.

<a href='#top'>Back to top</a>

<a id='section_2'></a>
## 2. Умножение матриц

**Задача:** Изменится ли качество линейной регрессии при умножении признаков на обратную матрицу?

**Ответ:** Качество линейной регрессии при умножении признаков на обратную матрицу не изменится. При умножении признаков на обратную матрицу, предсказания модели не поменяются, но поменяются веса для новых признаков.

**Обоснование:** Вектор весов линейной регрессии рассчитывается по формуле (3). Умножим признаки $X$ на обратную матрицу $P$ и посмотрим, изменится ли качество линейной регрессии. Представим новую матрицу признаков как $X_1$:

$$
\begin{equation}\tag{4} X_1 = X\cdot P\\ \end{equation}
$$

Подставим формулу (4) в формулу (3) и преобразуем уравнение:

\begin{equation}\tag{5}w_1 = ((X \cdot P)^T\cdot  X \cdot P)^{-1}\cdot  (X \cdot P)^T \cdot y\end{equation}

\begin{equation}\tag{6}w_1 = (P^T \cdot X^T\cdot X \cdot P)^{-1}\cdot  P^T \cdot X^T\cdot  y\end{equation}

\begin{equation}\tag{7}w_1 = P^{-1}\cdot  (X^T\cdot  X)^{-1} \cdot (P^T)^{-1}\cdot  P^T\cdot  X^T\cdot  y\end{equation}

Так как $P$ по условию обратная, то произведение $(P^T)^{-1}\cdot  P^T$ равно $E$ (единичная матрица):

$$
\begin{equation}\tag{8}w_1 = P^{-1} \cdot (X^T \cdot X)^{-1}\cdot  E\cdot  X^T \cdot y = P^{-1} \cdot ((X^T \cdot X)^{-1}\cdot  X^T \cdot y)\end{equation}
$$

Подставим в правую часть уравнения (8) формулу (3). Получим, что параметры линейной регрессии в исходной задаче и в преобразованной связаны следующим образом:

$$
\begin{equation}\tag{9}w_1 = P^{-1}\cdot  w\end{equation}
$$

Преобразуем формулу для предсказаний линейной регрессии (1). Подставим новое значение весов  $𝑤_1$, а также значения $X_1$ и $w_1$ для того, чтобы на их основе вычислить предсказания модели $a_1$:

$$
\begin{equation}\tag{10}a_1 = X_1\cdot w_1 = X\cdot P\cdot P^{-1}\cdot  w\end{equation}
$$

Так как $P$ по условию обратная, то произведение $P\cdot P^{-1}$ снова равно $E$:

$$
\begin{equation}\tag{11}a_1 = X\cdot P\cdot P^{-1} \cdot w = X\cdot  E\cdot  w = X\cdot  w = a\end{equation}
$$

#### Вывод
Доказано, что предсказания $a_1$ для матрицы признаков, умноженных на обратную матрицу $P$ равны предсказаниям $a$.

<a href='#top'>Back to top</a>

<a id='section_3'></a>
## 3. Алгоритм преобразования

**Алгоритм**

Для защиты данных без потери качества модели предложим алгоритм преобразования данных с созданием обратной матрицы $P$ и ее умножения на матрицу признаков.

**Обоснование**

Создадим обратную матрицу.

In [7]:
# Чтобы избежать получения вырожденной матрицы применим метод построения случайной матрицы randint()
invertible_matrix_rnd = np.random.randint(100, size = (4, 4))
invertible_matrix_rnd

array([[67, 20, 33, 11],
       [87, 64, 84, 89],
       [41, 51, 80, 85],
       [33, 19, 56,  1]])

In [8]:
# Проверим матрицу на обратимость 
matrix_inverted = np.linalg.inv(invertible_matrix_rnd)
matrix_inverted

array([[ 0.04172047, -0.02312419,  0.01901399, -0.01706184],
       [-0.17420245,  0.19159897, -0.1789049 ,  0.07083499],
       [ 0.03357634, -0.05037298,  0.04834457,  0.00456688],
       [ 0.05279622, -0.0563955 ,  0.06443542, -0.0385694 ]])

Матрица `invertible_matrix_rnd` обратима.

Расчитаем вектора предсказаний  $𝑎$  и  $𝑎_1$  по формулам и сравним их между собой.

In [9]:
# Выделим признаки для обучения и целевой признак
X = df.drop('insurance_compensations', axis=1)
y = df['insurance_compensations']
 
# Применим масштабирование
scaler = StandardScaler()
scaler.fit(X)

X_scaled = scaler.transform(X)

In [10]:
# Расчитаем вектор весов линейной регрессии (𝑤) по формуле (3)
w = np.linalg.inv(X_scaled.T@(X_scaled))@(X_scaled.T)@(y)

# Расчитаем вектор предсказаний (𝑎) по формуле (1)
a = X_scaled @ w

# Проведем преобразование признаков
X_1 = X_scaled @ invertible_matrix_rnd

# Расчитаем вектор весов линейной регрессии для преобразованных признаков (𝑤_1) по формуле (8)
w_1 = np.linalg.inv(X_1.T@(X_1))@(X_1.T)@(y)

# Расчитаем вектор предсказаний для преобразованных признаков (𝑎_1) по формуле (10)
a_1 = X_1 @ w_1

# Расчитаем расхождения элементов векторов (𝑎_1) и (a)
(a_1-a).sum()

2.4105522265782664e-13

#### Вывод
Расхождения минимальны $𝑎 \simeq 𝑎_1$, а значит алгоритм преобразования не приводит к разным предсказаниям модели и не ухудшает качество модели.

<a href='#top'>Back to top</a>

<a id='section_4'></a>
## 4. Проверка алгоритма

In [11]:
# Произведем проверку на модели линейной регрессии
model = LinearRegression(normalize = True)
model.fit(X, y)
predictions = model.predict(X)
mse = mean_squared_error(y, predictions)
print('MSE Score:', mse)
print('R2_Score: ',r2_score(y, predictions))

MSE Score: 0.12527263826681606
R2_Score:  0.4302010044852066


In [12]:
# Преобразуем признаки и повторим обучение
X_1 = X_scaled @ invertible_matrix_rnd

model.fit(X_1, y)
predictions_1 = model.predict(X_1)
mse = mean_squared_error(y, predictions_1)
print('MSE Score:', mse)
print('R2_Score: ',r2_score(y, predictions_1))

MSE Score: 0.12527263826681606
R2_Score:  0.4302010044852066


#### Вывод
Расхождения MSE и R2 обеих моделей минимальны, а значит данные пользователей защищены. При это качество модели сохранено.

<a href='#top'>Back to top</a>